## EmailWriting

In [19]:
import spacy
import json, re
from collections import Counter, defaultdict
from tqdm import tqdm

In [2]:
%%time
m = json.load(open('LBS_pattern.json', 'r'))
model = defaultdict(lambda: defaultdict(Counter))
for k, els in m.items():
    for p, els2 in els.items():
        for ng, count in els2.items():
            model[k][tuple(p.split())][ng] = int(count)


CPU times: user 10.1 s, sys: 950 ms, total: 11.1 s
Wall time: 11 s


In [3]:
nlp = spacy.load('en')
#doc = nlp(sent)

In [26]:
def find_pattern(cat:str, sentence:str, cmp_wd:int = 2, printable=True):
    top5 = []
    categ = model[cat]
    doc = nlp(sentence)
    tokens = [tok.lemma_ if '-' not in tok.lemma_ else 'SOMEONE' for tok in doc]
    #print(tokens)
    
    # 比對 sentence 和 ngram pattern是否一樣
    for p, els in categ.items():
        # 先比較 last 2 gram
        if len(tokens) >= 2 and tokens[-cmp_wd:] == list(p[:cmp_wd]):
            top5.append([sum(els.values()), ' '.join(p), list(els.items()), cmp_wd])
        # 再比較 last 1 gram
        elif tokens[-cmp_wd:] != list(p[:cmp_wd]) and tokens[-(cmp_wd-1):] == list(p[:cmp_wd-1]):
            top5.append([sum(els.values()), ' '.join(p), list(els.items()), cmp_wd-1])
        # 再比較 last 2-nd gram
        elif len(tokens) >= 2 and tokens[-cmp_wd:] != list(p[:cmp_wd]) and tokens[-(cmp_wd-1):] != list(p[:cmp_wd-1]) and tokens[-cmp_wd] == p[cmp_wd-2]:
            top5.append([sum(els.values()), ' '.join(p), list(els.items()), cmp_wd-1])
    
    #print(top5)
    # find top 5 pattern
    cat_pat = sorted(top5, key=lambda x: (-(x[3]*x[0])))
    # sort elements in each pattern by their counts
    for idx in range(len(cat_pat)):
        cat_pat[idx][2] = sorted(cat_pat[idx][2], key = lambda x: (-x[1], -(len(x[0].split()))))
    
    if not printable: return cat_pat
    
    print('category [%s]'%(cat.upper()))
    for total, pat, ng, _ in cat_pat[:5]:
        print()
        print('%s\t[%d]'%(pat, total))
        for idx in range(len(ng)):
            if idx == 3: break
            print('\t    [%s] %s\t[%d]'%(re.findall('\([A-Z]+\)', ng[idx][0])[0][1:-1], re.sub('\([A-Z]+\)', '', ng[idx][0]), ng[idx][1]))
        if len(ng) >=3 and idx == 3:
            print('\t         ...')
        print()
    

In [27]:
_ = find_pattern('suggestion', 'Traveling to have a business meeting takes')


category [SUGGESTION]

take out n.	[4]
	    [VERB] take out the home equity loan	[2]
	    [VERB] take out a loan	[1]
	    [VERB] take out bankruptcy	[1]


take n. to	[3]
	    [VERB] take a little time to	[1]
	    [VERB] take a moment to	[1]
	    [VERB] take care to	[1]


take SOMEONE to n.	[3]
	    [VERB] take you to lunch	[2]
	    [VERB] take them to a grocery store	[1]


take a n. to	[2]
	    [VERB] take a moment to	[1]
	    [VERB] take a minute to	[1]


take n. to v.	[2]
	    [VERB] take a moment to read	[1]
	    [VERB] take care to include	[1]



In [13]:
_ = find_pattern('acceptance', 'Mr. and fuck')

category [ACCEPTANCE]

and SOMEONE v.	[31]
	    [CCONJ] and I will	[16]
	    [CCONJ] and I look	[5]
	    [CCONJ] and I plan	[3]
	         ...


and I v.	[27]
	    [CCONJ] and I will	[16]
	    [CCONJ] and I look	[5]
	    [CCONJ] and I plan	[3]
	         ...


and v. n.	[15]
	    [CCONJ] and expect a long and rewarding association	[1]
	    [CCONJ] and organizing this company 's marketing branch	[1]
	    [CCONJ] and select a mutually acceptable time	[1]
	         ...


and SOMEONE v. be	[10]
	    [CCONJ] and I will be	[10]


and I v. be	[10]
	    [CCONJ] and I will be	[10]



### Eval

In [48]:
evals = json.load(open('eval.json','r'))
align = open('alignment.txt', 'r').read().strip().split('\n')
align = dict([a.split() for a in align])

In [32]:
score = []
hit = False
for k, v in tqdm(evals.items()):
    if align[k] != 'X':
        c = align[k]
        score.append([])
        for document in v:
            doc = nlp(document)
            tok, pos = list(zip(*[(tok.text, tok.pos_) for tok in doc]))
            sub_score, sub_total = 0, 0
            for idx in range(2, len(tok)-1):
                if pos[idx-1] == 'PUNCT' or pos[idx+1] == 'PUNCT':
                    continue
                else:
                    sug = find_pattern(c, ' '.join(tok[:idx]), printable=False)
                    if len(sug) == 0: continue
                    sub_total += 1
                    ans, ans2 = ' '.join(tok[idx-1:idx+1]), ' '.join(tok[idx-2:idx+1])
                    for num, (total, pat, ngs, _) in enumerate(sug):
                        for ng, _ in ngs:
                            ngram = re.sub('\([A-Z]+\)', '', ng)
                            if ans in ngram or ans2 in ngram:
                                sub_score += float(len(sug)-num) / float(len(sug)+1e-10)
                                hit = True
                                break
                        if hit:
                            hit = False
                            break
            score[-1].append(float(sub_score)/(float(sub_total)+1e-10))

100%|██████████| 50/50 [15:34<00:00, 18.69s/it]


In [53]:
score_dict = {}
idx = 0
for k, v in evals.items():
    if align[k] != 'X':
        score_dict[align[k]] = score[idx]
        idx+=1
        
with open('evaluation_score.json', 'w') as f:
    json.dump(score_dict, f)

### Eval Score

In [57]:
len(score), sum([sum(i) for i in score]) / sum([len(i) for i in score])

(29, 0.3166475321755383)